In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()["Outcome"].sort_values()

BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
DiabetesPedigreeFunction    0.173844
Pregnancies                 0.221898
Age                         0.238356
BMI                         0.292695
Glucose                     0.466581
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [9]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input

In [ ]:
model = Sequential(
    [Input(shape=(8,)), Dense(32, activation="relu"), Dense(1, activation="sigmoid")]
)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4129 - loss: 0.7707 - val_accuracy: 0.5130 - val_loss: 0.7023
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5291 - loss: 0.6993 - val_accuracy: 0.6364 - val_loss: 0.6441
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6027 - loss: 0.6483 - val_accuracy: 0.6818 - val_loss: 0.6050
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6438 - loss: 0.6193 - val_accuracy: 0.7013 - val_loss: 0.5754
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6673 - loss: 0.5955 - val_accuracy: 0.7208 - val_loss: 0.5540
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6928 - loss: 0.5747 - val_accuracy: 0.7208 - val_loss: 0.5352
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7279 - loss: 0.5339 - val_accuracy: 0.7273 - val_loss: 0.5212
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7124 - loss: 0.5238 - val_accuracy: 0.7403 

In [14]:
import kerastuner as kt

C:\Users\Muhammad Umer\AppData\Local\Temp\ipykernel_38272\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
from keras.src.optimizers import optimizer


def build_model(hp):

    model = Sequential()
    model.add(Dense(32, activation="relu", input_shape=(8,)))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice("optimzer", values=["adam", "sgd", "rmsprop", "adadelta"])
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=5)

C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.44155845046043396

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 18s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimzer': 'adam'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    initial_epoch=6,
    epochs=100,
    validation_data=(X_test, y_test),
)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.7256 - loss: 0.5205 - val_accuracy: 0.8052 - val_loss: 0.4925
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7368 - loss: 0.5223 - val_accuracy: 0.8182 - val_loss: 0.4815
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7447 - loss: 0.5043 - val_accuracy: 0.8117 - val_loss: 0.4738
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7473 - loss: 0.4924 - val_accuracy: 0.8052 - val_loss: 0.4704
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7414 - loss: 0.5041 - val_accuracy: 0.8052 - val_loss: 0.4656
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7781 - loss: 0.4620 - val_accuracy: 0.8052 - val_loss: 0.4621
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7875 - loss: 0.4572 - val_accuracy: 0.8117 - val_loss: 0.4604
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7847 - loss: 0.4513 - val_accurac

In [ ]:
def build_model(hp):
    model = Sequential()

    units = hp.Int("units", min_value=8, max_value=128, step=8)
    model.add(Dense(units, activation="relu", input_shape=(8,)))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="some_directory"
)

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 25s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    initial_epoch=6,
    epochs=100,
    validation_data=(X_test, y_test),
)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.7616 - loss: 0.4909 - val_accuracy: 0.7987 - val_loss: 0.4787
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7412 - loss: 0.5139 - val_accuracy: 0.7987 - val_loss: 0.4751
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7760 - loss: 0.4557 - val_accuracy: 0.8247 - val_loss: 0.4733
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7902 - loss: 0.4466 - val_accuracy: 0.8182 - val_loss: 0.4722
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7693 - loss: 0.4682 - val_accuracy: 0.8052 - val_loss: 0.4692
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7757 - loss: 0.4591 - val_accuracy: 0.8052 - val_loss: 0.4686
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8090 - loss: 0.4146 - val_accuracy: 0.8052 - val_loss: 0.4673
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7923 - loss: 0.4476 - val_accurac

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72, activation="relu", input_shape=(8,)))

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        model.add(Dense(72, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="mydir"
)

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 47s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(
    X_train, y_train, epochs=100, initial_epoch=5, validation_data=(X_test, y_test)
)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.7547 - loss: 0.4994 - val_accuracy: 0.7922 - val_loss: 0.5297
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7801 - loss: 0.4780 - val_accuracy: 0.8052 - val_loss: 0.5173
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8353 - loss: 0.3939 - val_accuracy: 0.7727 - val_loss: 0.4852
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7853 - loss: 0.4405 - val_accuracy: 0.7468 - val_loss: 0.5429
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8021 - loss: 0.4143 - val_accuracy: 0.7987 - val_loss: 0.4670
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8051 - loss: 0.4154 - val_accuracy: 0.7727 - val_loss: 0.4958
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8228 - loss: 0.4020 - val_accuracy: 0.7273 - val_loss: 0.6176
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8212 - loss: 0.4032 - val_accuracy

In [ ]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice(
                        "activation" + str(i), values=["relu", "tanh", "sigmoid"]
                    ),
                    input_shape=(8,),
                )
            )
        else:
            model.add(
                Dense(
                    hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice(
                        "activation" + str(i), values=["relu", "tanh", "sigmoid"]
                    ),
                )
            )
        counter += 1

    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=hp.Choice(
            "optimizer", values=["rmsprop", "adam", "sgd", "nadam", "adadelta"]
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [39]:
tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, directory="randomdir"
)

In [40]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 09s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 53s


In [41]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 24,
 'activation0': 'sigmoid',
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu'}

In [42]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Muhammad Umer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    initial_epoch=6,
    validation_data=(X_test, y_test),
)

Epoch 7/100


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7721 - loss: 0.4536 - val_accuracy: 0.7857 - val_loss: 0.4623
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7558 - loss: 0.4827 - val_accuracy: 0.8052 - val_loss: 0.4643
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7741 - loss: 0.4459 - val_accuracy: 0.7792 - val_loss: 0.4643
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8002 - loss: 0.4467 - val_accuracy: 0.8052 - val_loss: 0.4614
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7829 - loss: 0.4481 - val_accuracy: 0.8052 - val_loss: 0.4601
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7578 - loss: 0.4818 - val_accuracy: 0.7987 - val_loss: 0.4590
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7485 - loss: 0.4765 - val_accuracy: 0.7922 - val_loss: 0.4594
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7699 - loss: 0.4652 - val_accuracy: 0.7922 - val_loss